In [1]:
import polars as pl
import numpy as np
import time
from sklearn.svm import SVC
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, confusion_matrix, balanced_accuracy_score
from sklearn.model_selection import KFold
from imblearn.under_sampling import RandomUnderSampler

In [2]:
df_polars_raiz = pl.read_parquet('../dataset.parquet')

In [3]:
df_polars = df_polars_raiz.sample(fraction=0.01, seed=42)

In [4]:
import ipaddress

def ip_to_int(ip: str) -> int:
    try:
        return int(ipaddress.ip_address(ip))  # Funciona tanto para IPv4 quanto IPv6
    except ValueError:
        return None


In [5]:
#df_polars = df_polars.with_columns([
#    pl.col('id.resp_h').map_elements(ip_to_int).alias('id.resp_h'),
#    pl.col('id.orig_h').map_elements(ip_to_int).alias('id.orig_h')
#])

In [6]:
df_polars = df_polars.with_columns([
    pl.col('duration').fill_null(0),
    pl.col('orig_bytes').fill_null(0),
    pl.col('resp_bytes').fill_null(0)
])

In [7]:
lista_colunas = df_polars.columns
colunas_para_spearman = ['id.resp_p', 'history', 'conn_state', 'id.orig_p', 'orig_ip_bytes', 'label']             
#['detailed-label', 'id.resp_p', 'history', 'id.orig_h', 'conn_state', 'id.orig_p', 'orig_ip_bytes']
colunas_para_dropar = [col for col in lista_colunas if col not in colunas_para_spearman]
df_polars = df_polars.drop(colunas_para_dropar)

In [8]:
df_polars = df_polars.drop_nulls()

In [9]:
X = df_polars.drop('label')
y = df_polars['label']       

In [10]:
#X = X.to_numpy()
#X[:, 0] = np.array([ip_to_int(ip) for ip in X[:, 0]])

# Treinamento

In [11]:
results = []

In [12]:
def startTrain(X, y, kernel='rbf', C=1, gamma='scale', tol=0.001, max_iter=5000):    
    kfold = KFold(n_splits=5, shuffle=True, random_state=42)
    results_fold = []

    for train_idx, test_idx in kfold.split(X):
        X_train, X_test = X[train_idx], X[test_idx]
        y_train, y_test = y[train_idx], y[test_idx]
        
        scaler = MinMaxScaler()
        X_train_scaled = scaler.fit_transform(X_train)
        X_test_scaled = scaler.transform(X_test)
        
        rus = RandomUnderSampler(random_state=42)
        X_train_resampled, y_train_resampled = rus.fit_resample(X_train_scaled, y_train)

        svm = SVC(kernel=kernel, C=C, gamma=gamma, tol=tol, max_iter=max_iter)

        svm.fit(X_train_resampled, y_train_resampled)
        y_pred = svm.predict(X_test_scaled)

        accuracy = accuracy_score(y_test, y_pred)
        results_fold.append(accuracy)

    return np.mean(results_fold)

In [13]:
def grid_search(X, y):
    param_grid = {
        'kernel': ['sigmoid'],
        'C': [0.1, 1, 10, 100],
        'gamma': ['scale', 'auto', 0.01, 0.1, 1],
        'tol': [0.001, 0.0001],
        'max_iter': [5000, 10000]
    }

    best_accuracy = 0
    best_params = {}

    for kernel in param_grid['kernel']:
        for C in param_grid['C']:
            for gamma in param_grid['gamma']:
                for tol in param_grid['tol']:
                    for max_iter in param_grid['max_iter']:
                        
                        accuracy = startTrain(X, y, kernel=kernel, C=C, gamma=gamma, tol=tol, max_iter=max_iter)

                        print(f"kernel={kernel}, C={C}, gamma={gamma}, tol={tol}, max_iter={max_iter} -> Accuracy: {accuracy}")

                        if accuracy > best_accuracy:
                            best_accuracy = accuracy
                            best_params = {
                                'kernel': kernel,
                                'C': C,
                                'gamma': gamma,
                                'tol': tol,
                                'max_iter': max_iter
                            }

    print("\nMelhores Hiperparâmetros encontrados:", best_params)
    print(f"Melhor Acurácia Média: {best_accuracy}")
    return best_params

In [15]:
inicio = time.time()
best_params = grid_search(X,y)
fim = time.time()

c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\

kernel=sigmoid, C=0.1, gamma=scale, tol=0.001, max_iter=5000 -> Accuracy: 0.630641548595635


c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\

kernel=sigmoid, C=0.1, gamma=scale, tol=0.001, max_iter=10000 -> Accuracy: 0.6343372945759972


c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\

...

kernel=sigmoid, C=100, gamma=1, tol=0.001, max_iter=10000 -> Accuracy: 0.7396334381406202


c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=5000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\

kernel=sigmoid, C=100, gamma=1, tol=0.0001, max_iter=5000 -> Accuracy: 0.5836179970971396


c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(
c:\Users\rafae\anaconda3\envs\tcc_2025_py39\lib\site-packages\sklearn\svm\_base.py:297: ConvergenceWarning: Solver terminated early (max_iter=10000).  Consider pre-processing your data with StandardScaler or MinMaxScaler.
  warnings.warn(


kernel=sigmoid, C=100, gamma=1, tol=0.0001, max_iter=10000 -> Accuracy: 0.7396334381406202

Melhores Hiperparâmetros encontrados: {'kernel': 'sigmoid', 'C': 1, 'gamma': 0.01, 'tol': 0.001, 'max_iter': 10000}
Melhor Acurácia Média: 0.9844137497247841


In [16]:
duracao = fim - inicio
print(duracao)

33880.375868320465


In [17]:
print("\nMelhores Hiperparâmetros encontrados:", best_params)


Melhores Hiperparâmetros encontrados: {'kernel': 'sigmoid', 'C': 1, 'gamma': 0.01, 'tol': 0.001, 'max_iter': 10000}


In [18]:
for i in range(1,31):
    startTrain()
    print(i)

TypeError: startTrain() missing 2 required positional arguments: 'X' and 'y'

In [ ]:
metrics_df = pl.DataFrame(
    results,
    schema=['Algorithm', 'Accuracy', 'Balanced Accuracy' , 'Precision', 'Recall', 'Specificity', 'F1-score', 'False Alarm Rate', 'tn', 'fp', 'fn', 'tp', 'training_duration', 'evaluation_duration']
)
metrics_df

In [ ]:
metrics_df.write_csv("metrics_results/unbalanced_svm_metrics_output.csv", separator=';')